## Create Dataset for filtering: Idea Startup

In [ ]:
!pip3 install requests Pillow beautifulsoup4

In [ ]:
import requests, os
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://ideasai.com/data-startup-ideas")
soup = BeautifulSoup(response.text, "html.parser")

ideas = soup.findAll('h3', class_='idea')
idea_descriptions = [desc.text for desc in ideas]
print(idea_descriptions, len(idea_descriptions))

votes = soup.findAll('span', class_='votes')
voting = [v.text for v in votes]
print(voting)

print (len(voting), len(idea_descriptions))

import csv

file_path='data/valid_ideas.csv'
print(os.getcwd())

  # Try to open non-existing file to write data
with open(file_path, "x+") as file:
  # Read the contents of the file
  for idx in range(len(voting)):
    desc = idea_descriptions[idx].replace('"', '')
    vote = voting[idx].replace(',', '')
    sen = ';'.join([desc, '1', vote, '\n'])
    file.write(sen)

In [ ]:
import requests, json
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://randomwordgenerator.com/json/sentences.json")
type(response.text)
data = json.loads(response.text)
print(data["data"])
sentences = data["data"]

import csv

with open('data/invalid-sentences.csv', 'x+', newline='') as file:
  writer = csv.writer(file)
  
  for idx in range(len(sentences)):
    writer.writerow([sentences[idx]["sentence"], 0])
    

In [ ]:
import requests, json
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://www.randomsentencegen.com/sentence-with-startup")
soup = BeautifulSoup(response.text, "html.parser")

ideas = soup.findAll('tr', class_='item')
idea_descriptions = [desc.text for desc in ideas]
print(idea_descriptions, len(idea_descriptions))

import csv

with open('data/invalid-sentences.csv', 'x+', newline='') as file:
  writer = csv.writer(file)
  
  for idx in range(len(sentences)):
    writer.writerow([sentences[idx]["sentence"], 0])
    

## Translating En to Vie

In [ ]:
import pandas as pd

inv = pd.read_csv("data/invalid_sentences.csv", delimiter=';')
# invalid_1 = pd.read_csv("data/random_str.csv", delimiter=',')
semantic = pd.read_csv("data/valid_ideas.csv", delimiter=';', usecols=['sentence', 'spam'])

inv.head()
semantic.head()

frames = [inv, semantic]
 
df = pd.concat(frames)
en_sent = inv.sentence.to_list()
en_sent_1 = semantic.sentence.to_list()

en_sent

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import os

# Specify the file path
# file_path = "data/vie/text_valid.csv"

def write_to_file(sentence, label, file_path):
# Check if the file exists
  if os.path.exists(file_path):
      # File exists, open in append mode
      with open(file_path, "a") as file:
          file.write(f'{sentence};{label}\n')
  else:
      # File doesn't exist, create and write content
      with open(file_path, "w") as file:
          file.write(f'{sentence};{label}\n')


In [ ]:
write_to_file("hello", 1, 'data/vie/testttt.txt')

#### Write Vie SPAM Idea Dataset to CSV

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Specify the file path
file_path = "data/vie/spam.csv"

# Write Vie Spam Dataset to CSV
for sent in en_sent:
  sentence = "en: " + sent
  inputs = [sentence]
  outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids, max_length=512)
  vie_trans = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][4:]
  write_to_file(vie_trans,0, file_path)


#### Write Vie Valid Idea Description Dataset to CSV

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Specify the file path
file_path = "data/vie/valid_idea.csv"

# Write Vie Valid Idea Description Dataset to CSV
for sent in en_sent_1:
  sentence = "en: " + sent
  inputs = [sentence]
  outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids, max_length=512)
  vie_trans = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][4:]
  write_to_file(vie_trans,1,file_path)

## PhoBERT Embedding - Vietnamese Spam Filtering

[Hugging Face - PhoBERT](https://huggingface.co/docs/transformers/model_doc/phobert)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
  features = phobert(input_ids) # Models outputs are now tuples


In [ ]:
!pip3 install underthesea

In [ ]:
tag_sentence = 'Dịch vụ đặt vé trực tuyến của chúng tôi, EasyBooking, giúp bạn dễ dàng tìm kiếm, so sánh và đặt vé máy bay, khách sạn và tour du lịch. Với giao diện thân thiện và tính năng tiện lợi, chúng tôi mang đến trải nghiệm đặt vé nhanh chóng và tiết kiệm thời gian cho bạn'

In [ ]:
from underthesea import pos_tag, chunk, classify

pos_str = pos_tag(tag_sentence)
text_group = set([word for word,tag,num in chunk(tag_sentence) if tag == 'Np' or tag == 'N' or tag == 'V'])
# class_of_text = classify(tag_sentence)
nouns = set([word for word,tag in pos_str if tag == 'Np' or tag == 'N'])
print(text_group)

In [ ]:
!pip3 install gibberish-detector

In [ ]:
## Download https://metatext.io/datasets/cc100-vietnamese and merge all txt file text >> WIKI.txt

!gibberish-detector train data/WIKI.txt > vie.model

In [ ]:
from gibberish_detector import detector

# open('data/WIKI.txt')

Detector = detector.create_from_model('vie.model')
print(Detector.is_gibberish('Chào, tớ là osijd lldkn qwoie')) #Gibberish